In [106]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

# find product calories
def find_calories(page):
    table = page.find("table",class_=re.compile("product-info-nutrition_table*"))
    netto = [net.text for net in table.find("thead").find_all("th")[1:]]
    cals = []
    for td in table.find_all("td"):
        if "kcal" in td.text:
            calory = re.search("\d+ kcal|\d+kcal",td.text).group()
            cals.append(calory)
    calories = []
    for net,cal in zip(netto,cals):
        net = net.lower().replace("per","").strip()
        cal = "/".join([cal,net])
        calories.append(cal)
    return ", ".join(calories)

# find product ingredients
def find_ingredients(page):
    for h2 in page.find_all("h2"):
        if h2.text == "Ingrediënten":
            ingredients = h2.find_next().get_text()
            ingredients = re.sub("\s+"," ",ingredients)
            for w in re.findall("\.[A-Za-z]",ingredients):
                new_w = w.replace(".",". ")
                ingredients = ingredients.replace(w,new_w)
            try:
                a,b = re.search("Ingrediënten",ingredients).span()
                ingredients = ingredients.replace(ingredients[a:b],"").replace(":","")
                ingredients = ingredients.strip()
            except: pass
            if ingredients[-1] != ".":
                ingredients += "."
    return ingredients

In [107]:
url = "https://www.ah.nl/producten/product/wi54074/ah-komkommer"
page = render_html(url)

In [ ]:
product_data = {
    "code" : None,
    "sources" : None,
    "image_url" : None,
    "small_image_url" : None,
    "product_name" : None,
    "brand" : None,
    "nutriscore" : None,
    "categories" : None,
    "ingredients_text" : None,
    "calories" : None,
}

# find product data
for script in page.find_all("script",{"type":"application/ld+json"}):
    product = json.loads(script.get_text())
    type_ = product['@type']
    if type_ == "Product":
        product_data['code'] = product['gtin13']
        product_data['sources'] = [product['url']]
        product_data['small_image_url'] = product['image']
        product_data['product_name'] = product['name']
        product_data['brand'] = product['brand']['name']
    elif type_ == "BreadcrumbList":
        categories = []
        for item in product['itemListElement'][1:]:
            categories.append(item['item']['name'])
        product_data['categories'] = categories

# find product nutriscore
nutriscore = page.find("div",class_=re.compile("nutriscore*")).find("title").get_text().split(" ")[-1]
product_data['nutriscore'] = nutriscore

# find product image url
product_data['image_url'] = page.find("img",{"data-testhook":"product-image"})['src']

# find product calories
product_data['calories'] = find_calories(page)

# find product ingredients
product_data['ingredients'] = find_ingredients(page)

product_data